In [36]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter LangSmith API key")

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

from langchain_community.document_loaders import PyPDFLoader

# Load PDF
file_path = "./Data/coding_question.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()


print(len(docs))


print(f"{docs[0].page_content[:100]}\n")
print(docs[0].metadata)

#Splitting the document into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
texts = text_splitter.split_documents(docs) 

print("the number of splits" ,len(texts))


1
Coding Question and Sample Response
Question:
Write a Python function is_palindrome(s: str) -> bool 

{'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-02T21:12:10+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-02T21:12:10+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': './Data/coding_question.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}
the number of splits 9


In [37]:
import getpass
import os

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vector_1 = embeddings.embed_query(texts[0].page_content)

#assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])


Generated vectors of length 768

[0.029457075521349907, -0.013316336087882519, -0.013263420201838017, 0.0026578360702842474, -0.007666107267141342, 0.01710936799645424, 0.004043556749820709, 0.03881657496094704, -0.06280161440372467, -0.0033737951889634132]


In [38]:
# Vector Stores 

from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
ids = vector_store.add_documents(documents=texts)

# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("write a python function")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

print(results[0])


Score: 0.41184929626419836

page_content='Write a Python function is_palindrome(s: str) -> bool that determines whether a given string is a' metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-02T21:12:10+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-02T21:12:10+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': './Data/coding_question.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}
(Document(id='e6222a9a-fe9a-4e51-a6eb-3eaeb3f0fae6', metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-02T21:12:10+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-02T21:12:10+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': './Data/coding_question.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Write a Python function is_

In [ ]:
# Retrivers
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='1077ba44-0e40-4997-96a6-e5461f6f7a30', metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-02T21:12:10+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-02T21:12:10+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': './Data/coding_question.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Coding Question and Sample Response\nQuestion:')],
 [Document(id='15165400-db86-4653-bf4b-34742a3c2bf2', metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-02T21:12:10+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-02T21:12:10+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': './Data/coding_question.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='print(is_palindrome("Hello"))  # False')]]

In [ ]:
#Answers questions about a the provided pdf page. 

import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict



# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(docs.page_content for docs in state["context"])
    prompt = PromptTemplate.from_template("You are act as a reviewer agent for solving problem of in the domain of {docs_content}, When receive the question, compared with the ground truth as reference answer, and evaluate the answer and give a score over 100 as maximum. Also give the scorring details.")
    prompt_text = prompt.format(docs_content=docs_content)
    response = llm.invoke(prompt_text)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

response = graph.invoke({"question": "What is Task Decomposition?", "docs_content": "Coding Questions, Python "})
print(response["answer"])


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=9ea21dfb-3cca-4ddc-a21e-a64306575e7d,id=9ea21dfb-3cca-4ddc-a21e-a64306575e7d; trace=9ea21dfb-3cca-4ddc-a21e-a64306575e7d,id=8627a355-1582-4045-b5d7-0341f77bac1e; trace=9ea21dfb-3cca-4ddc-a21e-a64306575e7d,id=8627a355-1582-4045-b5d7-0341f77bac1e; trace=9ea21dfb-3cca-4ddc-a21e-a64306575e7d,id=ea9ce8ae-e93b-4182-9882-93918d63f074; trace=9ea21dfb-3cca-4ddc-a21e-a64306575e7d,id=5048f72e-252e-4422-9a7d-ce2829b0e7f1


As a reviewer agent, I have analyzed the provided "Sample Response" against the implied "Question" and the standards for a complete and correct solution.

**Question (Inferred):**
"Write a Python function `is_palindrome(s: str) -> bool` that checks if a given string `s` is a palindrome. The function should ignore spaces, punctuation, and capitalization when determining if the string is a palindrome."

**Ground Truth (Assumed for evaluation):**
A correct ground truth would include:
1.  A Python function `def is_palindrome(s: str) -> bool:`
2.  Implementation logic that:
    *   Converts the input string `s` to lowercase.
    *   Removes all non-alphanumeric characters (punctuation and spaces).
    *   Compares the cleaned string with its reverse.
3.  Correct example usage demonstrating the function.
4.  A clear and accurate definition of a palindrome.

**Review and Evaluation:**

The "Sample Response" provided is severely incomplete. It gives the function signature, an example usage, an

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=9ea21dfb-3cca-4ddc-a21e-a64306575e7d,id=5048f72e-252e-4422-9a7d-ce2829b0e7f1; trace=9ea21dfb-3cca-4ddc-a21e-a64306575e7d,id=ea9ce8ae-e93b-4182-9882-93918d63f074; trace=9ea21dfb-3cca-4ddc-a21e-a64306575e7d,id=9ea21dfb-3cca-4ddc-a21e-a64306575e7d
